In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import IPython
import subprocess
import time

In [2]:
endpointURL_reactome_v87 = "http://localhost:3030/reactome_v87/query"
endpointURL_reactome_pc = "http://localhost:3030/reactome_pc/query"
endpointURL_panther_pc = "http://localhost:3030/panther_pc/query"
endpointURL_pathbank_pc = "http://localhost:3030/pathbank_pc/query"
endpointURL_humancyc_pc = "http://localhost:3030/humancyc_pc/query"
endpointURL_kegg_pc = "http://localhost:3030/kegg_pc/query"
endpointURL_pid_pc = "http://localhost:3030/pid_pc/query"
endpointURL_inoh_pc = "http://localhost:3030/inoh_pc/query"
endpointURL_netpath_pc = "http://localhost:3030/netpath_pc/query"
rdfFormat = "turtle"

In [3]:
reactomeVersion = 87
prefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>

PREFIX chebi: <http://purl.obolibrary.org/obo/chebi/>
PREFIX chebidb: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX chebirel: <http://purl.obolibrary.org/obo/CHEBI#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

PREFIX bp3: <http://www.biopax.org/release/biopax-level3.owl#>

# Homo_sapiens-20170221.owl
#PREFIX reactome: <http://www.reactome.org/biopax/59/48887#> 
#
# Homo_sapiens-20210608.owl
#PREFIX reactome: <http://www.reactome.org/biopax/77/48887#>
#
# Homo_sapiens-20220614.owl
#PREFIX reactome: <http://www.reactome.org/biopax/81/48887#>
#
# Homo_sapiens-20221130.owl
#PREFIX reactome: <http://www.reactome.org/biopax/83/48887#>

PREFIX reactome: <http://www.reactome.org/biopax/{}/48887#>
""".format(reactomeVersion)

biopaxURI = "http://www.biopax.org/release/biopax-level3.owl#"

In [4]:
def displaySparqlResults(results):
    '''
    Displays as HTML the result of a SPARQLWrapper query in a Jupyter notebook.
    
        Parameters:
            results (dictionnary): the result of a call to SPARQLWrapper.query().convert()
    '''
    variableNames = results['head']['vars']
    tableCode = '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format('</th><th>'.join(variableNames), '</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join([row[vName]['value'] if vName in row.keys() else "&nbsp;" for vName in variableNames]))for row in results["results"]["bindings"]))
    IPython.display.display(IPython.display.HTML(tableCode))

## Mappings to ChEBI

In [5]:
# First SPARQL query using the string "ChEBI" to map to ChEBI resources
query1 = """
SELECT ?metric ?count
# Number of instances of SmallMolecule
WHERE {
    {
        SELECT ?metric (COUNT(DISTINCT ?small_molecule) AS ?count)
        WHERE {
            BIND("SmallMolecule" AS ?metric)
            OPTIONAL {
            ?small_molecule rdf:type/(rdfs:subClassOf*) bp3:SmallMolecule .
        }
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?small_molecule_ref) AS ?count)
        WHERE {
            BIND("SmallMoleculeReference" AS ?metric)
            OPTIONAL {
            ?small_molecule_ref rdf:type bp3:SmallMoleculeReference .
        }
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?small_molecule_without_ref) AS ?count)
        WHERE {
            BIND("noSMR" AS ?metric)
            OPTIONAL {
            ?small_molecule_without_ref rdf:type/(rdfs:subClassOf*) bp3:SmallMolecule .
            FILTER NOT EXISTS {
                ?small_molecule_without_ref bp3:entityReference ?ref .
                ?ref rdf:type bp3:SmallMoleculeReference .
            }
            }
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?small_molecule_that_map_to_chebi) AS ?count)
        WHERE {
            BIND("ChEBI mappings" AS ?metric)
            OPTIONAL {
            ?small_molecule_that_map_to_chebi rdf:type bp3:SmallMolecule .
            ?small_molecule_that_map_to_chebi bp3:entityReference ?small_molecule_ref .
            ?small_molecule_ref rdf:type bp3:SmallMoleculeReference .
            ?small_molecule_ref bp3:xref ?small_molecule_ref_xref .
            ?small_molecule_ref_xref rdf:type bp3:UnificationXref .
            ?small_molecule_ref_xref bp3:db "ChEBI" .
            ?small_molecule_ref_xref bp3:id ?chebi_id .
        }
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?small_molecule_no_map_to_chebi) AS ?count)
        WHERE {
            BIND("Smnomapping" AS ?metric)
            OPTIONAL {
            ?small_molecule_no_map_to_chebi rdf:type/(rdfs:subClassOf*) bp3:SmallMolecule .
            ?small_molecule_no_map_to_chebi bp3:entityReference ?small_molecule_ref .
            FILTER NOT EXISTS {
                ?small_molecule_ref bp3:xref ?xref .
                ?xref rdf:type bp3:UnificationXref .
                ?xref bp3:db "ChEBI" .
                ?xref bp3:id ?chebi_id .
            }
            }
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?chebi_id) AS ?count)
        WHERE {
            BIND("Unique ChEBI" AS ?metric)
            OPTIONAL {
            ?unif_xref rdf:type bp3:UnificationXref .
            ?unif_xref bp3:db "ChEBI" .
            ?unif_xref bp3:id ?chebi_id .
        }
        }
        GROUP BY ?metric
    }
}
ORDER BY ?metric

"""

# Second SPARQL query using the string "chebi" to map to ChEBI resources
query2 = """
SELECT ?metric ?count
# Number of instances of SmallMolecule
WHERE {
    {
        SELECT ?metric (COUNT(DISTINCT ?small_molecule) AS ?count)
        WHERE {
            BIND("SmallMolecule" AS ?metric)
            ?small_molecule rdf:type/(rdfs:subClassOf*) bp3:SmallMolecule .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?small_molecule_ref) AS ?count)
        WHERE {
            BIND("SmallMoleculeReference" AS ?metric)
            ?small_molecule_ref rdf:type bp3:SmallMoleculeReference .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?small_molecule_without_ref) AS ?count)
        WHERE {
            BIND("noSMR" AS ?metric)
            OPTIONAL {
            ?small_molecule_without_ref rdf:type/(rdfs:subClassOf*) bp3:SmallMolecule .
            FILTER NOT EXISTS {
                ?small_molecule_without_ref bp3:entityReference ?ref .
                ?ref rdf:type bp3:SmallMoleculeReference .
            }
            }
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?small_molecule_that_map_to_chebi) AS ?count)
        WHERE {
            BIND("ChEBI mappings" AS ?metric)
            ?small_molecule_that_map_to_chebi rdf:type bp3:SmallMolecule .
            ?small_molecule_that_map_to_chebi bp3:entityReference ?small_molecule_ref .
            ?small_molecule_ref rdf:type bp3:SmallMoleculeReference .
            ?small_molecule_ref bp3:xref ?small_molecule_ref_xref .
            ?small_molecule_ref_xref rdf:type bp3:UnificationXref .
            ?small_molecule_ref_xref bp3:db "chebi" .
            ?small_molecule_ref_xref bp3:id ?chebi_id .
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?small_molecule_no_map_to_chebi) AS ?count)
        WHERE {
            BIND("Smnomapping" AS ?metric)
            ?small_molecule_no_map_to_chebi rdf:type/(rdfs:subClassOf*) bp3:SmallMolecule .
            ?small_molecule_no_map_to_chebi bp3:entityReference ?small_molecule_ref .
            FILTER NOT EXISTS {
                ?small_molecule_ref bp3:xref ?xref .
                ?xref rdf:type bp3:UnificationXref .
                ?xref bp3:db "chebi" .
                ?xref bp3:id ?chebi_id .
            }
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?chebi_id) AS ?count)
        WHERE {
            BIND("Unique ChEBI" AS ?metric)
            ?unif_xref rdf:type bp3:UnificationXref .
            ?unif_xref bp3:db "chebi" .
            ?unif_xref bp3:id ?chebi_id .
        }
        GROUP BY ?metric
    }
}
ORDER BY ?metric

"""

TODO: change the path to fuseki server and BioPAX files + BioPAX ontology in the command line to launch the SPARQL endpoint

#### Query on Reactome BioPAX standalone export version 87

In [6]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/ReactomeBioPAX/_00_Reactome_Data_v87/Homo_sapiens.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/reactome_v87']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_reactome_v87)
sparql.setQuery(prefixes+query1)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open(f"../Results/reactome_v87_mappings_chebi.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

09:27:04 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/ReactomeBioPAX/_00_Reactome_Data_v87/Homo_sapiens.owl
09:27:05 WARN  riot            :: [line: 67280, col: 52] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
09:27:25 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
09:27:25 INFO  Server          :: Running in read-only mode for /reactome_v87
09:27:25 INFO  Server          :: Apache Jena Fuseki 4.9.0
09:27:25 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
09:27:25 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run
09:27:25 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run/shiro.ini
09:27:25 INFO  Server   

metric,count
ChEBI mappings,3645
SmallMolecule,5196
SmallMoleculeReference,2956
Smnomapping,1045
Unique ChEBI,2000
noSMR,506


09:28:04 INFO  Fuseki          :: [2] GET http://localhost:3030/reactome_v87/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.rea

#### Query on BioPAX export of Reactome from PathwayCommons

In [7]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.reactome.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/reactome_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_reactome_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open(f"../Results/reactome_pc_mappings_chebi.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

09:29:04 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.reactome.BIOPAX.owl
09:29:06 WARN  riot            :: [line: 100212, col: 81] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
09:29:26 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
09:29:26 INFO  Server          :: Running in read-only mode for /reactome_pc
09:29:26 INFO  Server          :: Apache Jena Fuseki 4.9.0
09:29:26 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
09:29:26 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run
09:29:26 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run/shiro.ini
09:29:27 INFO  Server

metric,count
ChEBI mappings,3522
SmallMolecule,3983
SmallMoleculeReference,2289
Smnomapping,461
Unique ChEBI,1960
noSMR,0


09:30:05 INFO  Fuseki          :: [2] GET http://localhost:3030/reactome_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reac

#### Query on BioPAX export of Panther from PathwayCommons

In [8]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.panther.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/panther_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_panther_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/panther_pc_mappings_chebi.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

09:31:05 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.panther.BIOPAX.owl
09:31:07 WARN  riot            :: [line: 80968, col: 73] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
09:31:09 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
09:31:09 INFO  Server          :: Running in read-only mode for /panther_pc
09:31:09 INFO  Server          :: Apache Jena Fuseki 4.9.0
09:31:09 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
09:31:09 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run
09:31:09 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run/shiro.ini
09:31:09 INFO  Server   

metric,count
ChEBI mappings,657
SmallMolecule,1936
SmallMoleculeReference,1458
Smnomapping,1279
Unique ChEBI,179
noSMR,0


09:32:05 INFO  Fuseki          :: [2] GET http://localhost:3030/panther_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.react

#### Query on BioPAX export of PathBank from PathwayCommons

In [9]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.pathbank.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/pathbank_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_pathbank_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/pathbank_pc_mappings_chebi.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

09:33:06 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.pathbank.BIOPAX.owl
09:33:08 WARN  riot            :: [line: 89570, col: 87] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
09:33:09 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
09:33:10 INFO  Server          :: Running in read-only mode for /pathbank_pc
09:33:10 INFO  Server          :: Apache Jena Fuseki 4.9.0
09:33:10 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
09:33:10 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run
09:33:10 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run/shiro.ini
09:33:10 INFO  Server 

metric,count
ChEBI mappings,3012
SmallMolecule,3326
SmallMoleculeReference,1776
Smnomapping,314
Unique ChEBI,1601
noSMR,0


09:34:06 INFO  Fuseki          :: [2] GET http://localhost:3030/pathbank_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reac

#### Query on BioPAX export of HumanCyc from PathwayCommons

In [10]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.humancyc.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/humancyc_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_humancyc_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/humancyc_pc_mappings_chebi.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

09:35:07 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.humancyc.BIOPAX.owl
09:35:09 WARN  riot            :: [line: 108021, col: 73] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
09:35:12 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
09:35:12 INFO  Server          :: Running in read-only mode for /humancyc_pc
09:35:12 INFO  Server          :: Apache Jena Fuseki 4.9.0
09:35:12 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
09:35:12 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run
09:35:12 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run/shiro.ini
09:35:12 INFO  Server

metric,count
ChEBI mappings,1705
SmallMolecule,2174
SmallMoleculeReference,1925
Smnomapping,469
Unique ChEBI,1473
noSMR,0


09:36:07 INFO  Fuseki          :: [2] GET http://localhost:3030/humancyc_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reac

#### Query on BioPAX export of KEGG from PathwayCommons

In [11]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.kegg.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/kegg_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_kegg_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/kegg_pc_mappings_chebi.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

09:37:08 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.kegg.BIOPAX.owl
09:37:09 WARN  riot            :: [line: 96269, col: 75] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
09:37:11 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
09:37:11 INFO  Server          :: Running in read-only mode for /kegg_pc
09:37:11 INFO  Server          :: Apache Jena Fuseki 4.9.0
09:37:11 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
09:37:11 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run
09:37:11 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run/shiro.ini
09:37:11 INFO  Server         

metric,count
ChEBI mappings,1465
SmallMolecule,1664
SmallMoleculeReference,1644
Smnomapping,199
Unique ChEBI,1445
noSMR,0


09:38:08 INFO  Fuseki          :: [2] GET http://localhost:3030/kegg_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome

#### Query on BioPAX export of PID from PathwayCommons

In [12]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.pid.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/pid_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_pid_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/pid_pc_mappings_chebi.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

09:39:08 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.pid.BIOPAX.owl
09:39:10 WARN  riot            :: [line: 77671, col: 87] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
09:39:14 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
09:39:14 INFO  Server          :: Running in read-only mode for /pid_pc
09:39:14 INFO  Server          :: Apache Jena Fuseki 4.9.0
09:39:14 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
09:39:14 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run
09:39:14 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run/shiro.ini
09:39:15 INFO  Server          :

metric,count
ChEBI mappings,113
SmallMolecule,188
SmallMoleculeReference,123
Smnomapping,75
Unique ChEBI,80
noSMR,0


09:40:08 INFO  Fuseki          :: [2] GET http://localhost:3030/pid_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.

#### Query on BioPAX export of INOH from PathwayCommons

In [13]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.inoh.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/inoh_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_inoh_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/inoh_pc_mappings_chebi.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

09:41:09 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.inoh.BIOPAX.owl
09:41:10 WARN  riot            :: [line: 100079, col: 94] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
09:41:16 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
09:41:16 INFO  Server          :: Running in read-only mode for /inoh_pc
09:41:16 INFO  Server          :: Apache Jena Fuseki 4.9.0
09:41:17 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
09:41:17 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run
09:41:17 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run/shiro.ini
09:41:17 INFO  Server        

metric,count
ChEBI mappings,3423
SmallMolecule,4273
SmallMoleculeReference,529
Smnomapping,850
Unique ChEBI,439
noSMR,0


09:42:09 INFO  Fuseki          :: [2] GET http://localhost:3030/inoh_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome

#### Query on BioPAX export of NetPath from PathwayCommons

In [14]:
# TODO: change path
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.netpath.BIOPAX.owl',
    '--file', '/home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl',
    '/netpath_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_netpath_pc)
sparql.setQuery(prefixes+query1)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/netpath_pc_mappings_chebi.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

09:43:10 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/Compare_BioPAX_Files/PathwayCommonsFiles/PathwayCommons12.netpath.BIOPAX.owl
09:43:11 WARN  riot            :: [line: 83771, col: 87] {W137} Input is large. Switching off checking for illegal reuse of rdf:ID's.
09:43:12 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAX_Ontology/biopax-level3.owl
09:43:12 INFO  Server          :: Running in read-only mode for /netpath_pc
09:43:12 INFO  Server          :: Apache Jena Fuseki 4.9.0
09:43:12 INFO  Config          :: FUSEKI_HOME=/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0
09:43:12 INFO  Config          :: FUSEKI_BASE=/home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run
09:43:12 INFO  Config          :: Shiro file: file:///home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/run/shiro.ini
09:43:13 INFO  Server   

metric,count
ChEBI mappings,0
SmallMolecule,0
SmallMoleculeReference,0
Smnomapping,0
Unique ChEBI,0
noSMR,0


09:44:09 INFO  Fuseki          :: [2] GET http://localhost:3030/netpath_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.react